In [1]:
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd  /content/drive/Shared\ drives/256/ASKus

/content/drive/Shared drives/256/ASKus


In [ ]:
# import pandas as pd

# train = pd.read_csv('./data/train.csv')
# train

In [4]:
import json, re

# SPACY_TRAIN = []
LABEL = 'data'

# count = 0
# for index, row in train.iterrows():
#   print(count)
#   this_pub = ""
#   this_pub_id = './data/train/' + row['Id'] + '.json'
#   this_data_lab = row['dataset_label']
#   this_pub_json = json.load(open(this_pub_id))
#   for section in this_pub_json:
#     this_pub += section['text']
#   this_ent_dict = {"entities": []}
#   for m in re.finditer(this_data_lab, this_pub):
#     this_ent_dict['entities'].append((m.start(), m.end()+1, LABEL))
#   SPACY_TRAIN.append((this_pub, this_ent_dict))
#   count+=1

# SPACY_TRAIN

In [ ]:
# file = open("./data/spacy_train.txt", "w")
# SPACY_TRAIN_File = repr(SPACY_TRAIN)
# file.write(SPACY_TRAIN_File)
# file.close()

In [5]:
# import module
import fileinput
import time
  
#time at the start of program is noted
start = time.time()
spacy_train = ""
#keeps a track of number of lines in the file
count = 0
for lines in fileinput.input(["./data/spacy_train.txt"]):
    spacy_train = lines
    count = count + 1
      
#time at the end of program execution is noted
end = time.time()
  
#total time taken to print the file
print("Execution time in seconds: ",(end - start))
print("No. of lines printed: ",count)
print(len(spacy_train))

Execution time in seconds:  6.801433324813843
No. of lines printed:  1
1093591763


In [6]:
import ast

DATA = ast.literal_eval(spacy_train)

print(len(DATA))

19661


In [71]:
new_spacy = []

for paper in DATA:
  hit_count = 0
  miss_count = 0
  entities = paper[1]['entities']
  if len(entities) > 0:
    dataset = paper[0][entities[0][0]:entities[0][1]-1]

  sentences = paper[0].split('.')
  for sentence in sentences:
    this_ent_dict = {"entities": []}
    if dataset in sentence:
      hit_count +=1
      for m in re.finditer(dataset,sentence):
        this_ent_dict['entities'].append((m.start(), m.end()+1, LABEL))
      new_spacy.append((sentence, this_ent_dict))
    elif miss_count < hit_count:
      miss_count += 1
      new_spacy.append((sentence, this_ent_dict))
    if hit_count >= 1 and miss_count >= 1:
      break

In [72]:
print(len(new_spacy))

31687


In [70]:
print(new_spacy[0])

[('This study used data from the National Education Longitudinal Study (NELS:88) to examine the effects of dual enrollment programs for high school students on college degree attainment', {'entities': [(30, 68, 'data')]}), (' The study also reported whether the impacts of dual enrollment programs were different for first generation college students versus students whose parents had attended at least some college', {'entities': []}), ('The National Education Longitudinal Study of 1988 (NELS:88) is a longitudinal survey that began in the spring of 1988 with a cohort of 24,599 8th graders', {'entities': [(4, 42, 'data')]}), ('A study investigated the college application behaviors of students from different racial/ethnic groups (Whites, African Americans, Asian Americans, Hispanic Americans) to understand differences in the college search and choice process', {'entities': []}), ('Data from the National Education Longitudinal Study: 1988Study: -94 (1996 was used to examine participation in 

In [73]:
count = 0
sen = 0
while sen < len(new_spacy):
  if new_spacy[sen][0].strip():
      # print(new_spacy[sen])
      sen += 1
  else:
      # print(new_spacy[sen])
      del new_spacy[sen]
      count+=1
print(count)

31


In [74]:
import random
random.shuffle(new_spacy)

train_mark = (len(new_spacy)*7//10)
TRAIN_DATA = new_spacy[:train_mark]
TEST_DATA = new_spacy[train_mark:]

In [75]:
# for sen in TRAIN_DATA:
#     if sen[0].strip():
#       pass
#     else:
#       print(sen)
print(len(TRAIN_DATA))

22159


In [78]:
import spacy as sp
from spacy.util import minibatch, compounding

nlp = sp.blank('en')
print("created blank english model")

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

ner.add_label(LABEL)
optimizer = nlp.begin_training()

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(3):
        print(itn)
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=500)
        for batch in batches:
            print('batch', len(batch))
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                        losses=losses)
        print('Losses', losses)

created blank english model
0
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 159
Losses {'ner': 68596.64136886597}
1
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 500
batch 159
Losses {'ner': 5065.090503692627}


In [18]:
test_text = TEST_DATA[0]
print(test_text[1])

{'entities': [(31, 52, 'data')]}


In [80]:
for test_text in TEST_DATA:
  doc = nlp(test_text[0])
  # print("Entities in '%s'" % test_text)
  count = 0
  if len(doc.ents) > 0:
    count += len(doc.ents)
print(count)
    # for ent in doc.ents:
    #     print(ent.label_, ent.text)

KeyboardInterrupt: ignored

In [86]:
import json

test = json.load(open('./data/test/8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60.json'))

this_pub = ""
for section in test:
  this_pub += section['text']

sentences = this_pub.split('.')
for sentence in sentences:
  doc = nlp(sentence)
  # print("Entities in '%s'" % test_text)
  if len(doc.ents) > 0:
    for ent in doc.ents:
        print(ent.label_, ent.text)


# count = 0
# for index, row in train.iterrows():
#   print(count)
#   this_pub = ""
#   this_pub_id = './data/train/' + row['Id'] + '.json'
#   this_data_lab = row['dataset_label']
#   this_pub_json = json.load(open(this_pub_id))
#   for section in this_pub_json:
#     this_pub += section['text']
#   this_ent_dict = {"entities": []}
#   for m in re.finditer(this_data_lab, this_pub):
#     this_ent_dict['entities'].append((m.start(), m.end()+1, LABEL))
#   SPACY_TRAIN.append((this_pub, this_ent_dict))
#   count+=1

# SPACY_TRAIN

data Rural-Urban Continuum Codes (


In [ ]:
this_pub = ""
for section in data:
  this_pub += section['text']


In [ ]:
import re
this_data_lab = "National Education Longitudinal Study"
for m in re.finditer(this_data_lab, this_pub):
  this_pub[m.start():m.end()+1]
print(this_pub[2937:2975])
#[0, 5, 10, 15]

ely to attain a college degree than co


In [ ]:
# !pip install -U spacy
# !python -m spacy download en

#check num json files in train (should be 14,316)
# import glob
# len(glob.glob1("./data/train", "*.json"))